In [1]:
specifics = 'calculated_limb_calculated_alpha'
path = "current_fits"

In [2]:
from automatic_calculated_limb_calculated_alpha import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from math import *
from datetime import datetime

In [3]:
sweet_cat_parameters = pd.read_csv("all_spectra_parameters.csv")
sweet_cat_parameters.astype({'vmic': 'float', 'err_vmic': 'float'}).dtypes
sweet_cat_parameters

,ind,Spectra,SNR,Resolution,Star,Star_SC,teff,err_teff,logg,err_logg,vmic,err_vmic,MH,err_MH,vsini,err_up_vsini,err_down_vsini
0,0,HD19994A_HARPSS_115000_378_691_2020.fits,2021.85,115000,HD19994A,HD 19994 A,6249,27,4.26,0.04,1.57,0.03,0.27,0.02,8.60,1.720,-1.720
1,1,HD60532_HARPSS_115000_378_691_2020.fits,1940.86,115000,HD60532,HD 60532,6281,34,4.00,0.04,1.80,0.04,-0.07,0.02,8.00,1.600,-1.600
2,2,HR858_HARPSS_115000_378_691_2020.fits,1772.89,115000,HR858,HR 858,6360,26,4.50,0.04,1.51,0.04,-0.02,0.02,8.30,0.500,-0.500
3,3,HD10647_HARPSS_115000_378_691_2020.fits,1696.77,115000,HD10647,HD 10647,6178,20,4.49,0.03,1.26,0.03,-0.01,0.01,4.90,0.980,-0.980
4,4,HD208487_HARPSS_115000_378_691_2020.fits,1461.86,115000,HD208487,HD 208487,6137,18,4.46,0.03,1.21,0.03,0.09,0.01,4.61,0.922,-0.922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,90,WASP-135_SOPHIE_HE_2020.fits,82.53,40000,WASP-135,WASP-135,5052,77,4.11,0.22,0.85,0.14,0.17,0.05,4.67,0.890,-0.890
91,91,WASP-139_HARPSS_115000_378_691_2020.fits,56.31,115000,WASP-139,WASP-139,5215,38,4.35,0.08,0.82,0.07,0.12,0.03,4.20,1.100,-1.100
92,92,WASP-150_SOPHIE_HE_2020.fits,127.76,40000,WASP-150,WASP-150,6486,48,4.46,0.09,1.61,0.06,0.27,0.03,8.82,0.950,-0.950
93,93,WASP-173A_HARPSS_115000_378_691_2020.fits,110.24,115000,WASP-173A,KELT-22A,5803,26,4.45,0.04,1.29,0.03,0.21,0.02,6.10,0.300,-0.300


In [4]:

outliers = ['HD66141', 'KPS-1', 'WASP-92','Kepler-41', 'CoRoT-4', 'HD133131B', 'HATS-3','WASP-87A', 'Kepler-40', 'Kepler-435']
 

In [5]:
#appending 'HATS-10' and 'HATS-56' to outliers because they have caused problems
if 'HATS-10' not in outliers:
    outliers.append('HATS-10')
if 'HATS-56' not in outliers:
    outliers.append('HATS-56')
print(len(outliers))

12


# HD66141

## Turbospectrum 2

In [6]:
line_list = "2"
code = "turbospectrum"
star_name = 'HD66141'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

[2023-07-27 19:50:41,124] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:50:41,128] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:50:41,130] [INFO] [spectrum:estimate_snr:451]: 0.00%


28


[2023-07-27 19:50:41,625] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:50:42,123] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:50:42,645] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:50:43,150] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:50:43,656] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:50:44,170] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:50:44,673] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:50:45,173] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:50:45,659] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:50:46,164] [INFO] [spectrum:estimate_snr:458]: SNR = 174.80
[2023-07-27 19:50:46,242] [INFO] [continuum:__fit_continuum:637]: Resampling spectrum to wave_step: 0.00100 nm (R ~ 472978)
[2023-07-27 19:50:48,724] [INFO] [continuum:__median_filter:307]: Median filter with step: 51
[2023-07-27 19:50:48,843] [INFO] [continuum:__max_filter:298]: Max filter with step: 4

Generating: 4357 1.99 -0.33 0.13 1.53 4.61 2.00 0.60 42310 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  9500.449099  DOF =  6778
   teff = 4357  
   logg = 1.99  
   MH = -0.33  
   alpha = 0.1391  
   Vmic = 1.53  
   Vmac = 4.61  
   Vsini = 2  
   limb_darkening_coeff = 0.6  
   R = 42310  
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 4.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []


[2023-07-27 19:53:58,452] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:53:58,453] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    1.06	  0.0482	    0.05	  0.0022	  0.0462
Line        489.15	  489.13	  489.21	   16.05	  0.1957	    0.24	  0.0029	  0.0540
Line        490.33	  490.28	  490.37	    4.99	  0.0616	    0.06	  0.0008	  0.0277
Line        492.39	  492.34	  492.42	   11.57	  0.1522	    0.14	  0.0019	  0.0431
Line        492.48	  492.45	  492.51	   25.10	  0.4115	    0.19	  0.0032	  0.0559
Line        493.88	  493.86	  493.91	   12.35	  0.2573	    0.10	  0.0021	  0.0453
Line        496.26	  496.24	  496.28	    5.45	  0.1434	    0.06	  0.0015	  0.0378
Line        496.61	  496.59	  496.65	   55.90	  1.0961	    0.44	  0.0086	  0.0921
Line        499.41	  499.39	  499.44	   60.93	  1.2186	    0.13	  0.0025	  0.0500
Line        500.19	  500.16	  500.22	   12.86	  0.2041	    0.10	  0.0016	  0.0401
Line        501.49	  501.45	  501.52	   31.42	  0.4909	    0.28	  0.0044	  0.0657
Line        

/tmp/ipykernel_1748722/2630082441.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


109

## Synthe 2

In [7]:
line_list = "2"
code = "synthe"
star_name = 'HD66141'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

28


[2023-07-27 19:53:58,620] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:53:58,624] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:53:58,627] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:53:59,164] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:53:59,732] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:54:00,354] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:54:00,937] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:54:01,509] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:54:02,057] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:54:02,615] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:54:03,179] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:54:03,731] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:54:04,293] [INFO] [spectrum:estimate_snr:458

Generating: 4357 1.99 -0.33 0.13 1.53 4.61 2.00 0.60 42310 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  9566.54928  DOF =  6778
   teff = 4357  
   logg = 1.99  
   MH = -0.33  
   alpha = 0.1391  
   Vmic = 1.53  
   Vmac = 4.61  
   Vsini = 2  
   limb_darkening_coeff = 0.6  
   R = 42310  
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 4.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []


[2023-07-27 19:57:12,663] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:57:12,666] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.83	  0.0375	    0.04	  0.0017	  0.0404
Line        489.15	  489.13	  489.21	   15.74	  0.1920	    0.24	  0.0029	  0.0538
Line        490.33	  490.28	  490.37	    4.80	  0.0593	    0.07	  0.0009	  0.0296
Line        492.39	  492.34	  492.42	   14.46	  0.1902	    0.17	  0.0023	  0.0473
Line        492.48	  492.45	  492.51	   21.16	  0.3469	    0.17	  0.0028	  0.0522
Line        493.88	  493.86	  493.91	   17.52	  0.3651	    0.14	  0.0030	  0.0542
Line        496.26	  496.24	  496.28	    4.45	  0.1170	    0.05	  0.0012	  0.0343
Line        496.61	  496.59	  496.65	   51.13	  1.0026	    0.42	  0.0082	  0.0896
Line        499.41	  499.39	  499.44	   60.43	  1.2087	    0.13	  0.0026	  0.0508
Line        500.19	  500.16	  500.22	   18.63	  0.2957	    0.14	  0.0023	  0.0476
Line        501.49	  501.45	  501.52	   32.75	  0.5117	    0.29	  0.0045	  0.0667
Line        

/tmp/ipykernel_1748722/4231307687.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


102

## Moog 2

In [8]:
line_list = "2"
code = "moog"
star_name = 'HD66141'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

28


[2023-07-27 19:57:12,911] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:57:12,919] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:57:12,923] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:57:13,710] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:57:14,542] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:57:15,307] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:57:16,054] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:57:16,842] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:57:17,597] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:57:18,331] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:57:19,135] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:57:20,083] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:57:20,864] [INFO] [spectrum:estimate_snr:458

Generating: 4357 1.99 -0.33 0.13 1.53 4.61 2.00 0.60 42310 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  9872.673302  DOF =  6778
   teff = 4357  
   logg = 1.99  
   MH = -0.33  
   alpha = 0.1391  
   Vmic = 1.53  
   Vmac = 4.61  
   Vsini = 2  
   limb_darkening_coeff = 0.6  
   R = 42310  
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 4.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []


[2023-07-27 19:58:09,772] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:58:09,774] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    1.15	  0.0521	    0.05	  0.0024	  0.0482
Line        489.15	  489.13	  489.21	   14.91	  0.1818	    0.28	  0.0034	  0.0582
Line        490.33	  490.28	  490.37	    5.57	  0.0688	    0.05	  0.0006	  0.0243
Line        492.39	  492.34	  492.42	   24.51	  0.3225	    0.31	  0.0041	  0.0639
Line        492.48	  492.45	  492.51	   31.02	  0.5086	    0.24	  0.0040	  0.0624
Line        493.88	  493.86	  493.91	   14.22	  0.2963	    0.10	  0.0021	  0.0450
Line        496.26	  496.24	  496.28	    5.52	  0.1453	    0.06	  0.0015	  0.0376
Line        496.61	  496.59	  496.65	   61.48	  1.2055	    0.48	  0.0094	  0.0961
Line        499.41	  499.39	  499.44	   50.62	  1.0123	    0.11	  0.0023	  0.0471
Line        500.19	  500.16	  500.22	   10.45	  0.1659	    0.07	  0.0011	  0.0330
Line        501.49	  501.45	  501.52	   43.26	  0.6759	    0.38	  0.0060	  0.0766
Line        

/tmp/ipykernel_1748722/3433610943.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


100

# WASP-92

## Turbospectrum 2

In [9]:
line_list = "2"
code = "turbospectrum"
star_name = 'WASP-92'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

94


[2023-07-27 19:58:10,107] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:58:10,117] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:58:10,123] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:58:11,007] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:58:11,927] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:58:12,731] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:58:13,468] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:58:14,211] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:58:14,951] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:58:15,710] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:58:16,503] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:58:17,248] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:58:18,068] [INFO] [spectrum:estimate_snr:458

Generating: 6039 4.21 0.10 0.00 0.95 4.42 2.00 0.59 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  106.1107468  DOF =  3456
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 2  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 4.00 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 25.04 0.59 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  99.44816304  DOF =  3456
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 25.04217638  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 27.04 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 0.59 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 10.41 0.59 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  99.06865739  DOF =  3456

[2023-07-27 20:03:39,014] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:03:39,016] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.62	  0.0415	    2.39	  0.1593	  0.3864
Line        489.15	  489.13	  489.21	    1.08	  0.0245	    4.57	  0.1039	  0.3188
Line        490.33	  490.28	  490.37	    1.99	  0.0343	   11.51	  0.1984	  0.4416
Line        492.39	  492.34	  492.42	    1.40	  0.0351	    7.39	  0.1847	  0.4245
Line        492.48	  492.45	  492.51	    0.97	  0.0303	    3.91	  0.1222	  0.3443
Line        493.88	  493.86	  493.91	    0.83	  0.0246	    5.54	  0.1630	  0.3979
Line        496.26	  496.24	  496.28	    0.82	  0.0431	    3.41	  0.1796	  0.4131
Line        496.61	  496.59	  496.65	    0.97	  0.0374	    4.94	  0.1900	  0.4277
Line        499.41	  499.39	  499.44	    0.96	  0.0319	    4.76	  0.1585	  0.3917
Line        500.19	  500.16	  500.22	    0.93	  0.0290	    4.77	  0.1491	  0.3802
Line        501.49	  501.45	  501.52	    1.26	  0.0314	    4.75	  0.1188	  0.3405
Line        

160

## Synthe 2

In [10]:
line_list = "2"
code = "synthe"
star_name = 'WASP-92'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

94


[2023-07-27 20:03:39,209] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:03:39,214] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:03:39,216] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:03:39,754] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:03:40,318] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:03:40,966] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:03:41,552] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:03:42,115] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:03:42,688] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:03:43,243] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:03:43,796] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:03:44,396] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:03:44,998] [INFO] [spectrum:estimate_snr:458

Generating: 6039 4.21 0.10 0.00 0.95 4.42 2.00 0.59 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  104.211085  DOF =  3447
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 2  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 4.00 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 25.53 0.59 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  97.23105852  DOF =  3447
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 25.53262635  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 27.53 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 0.00 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 10.93 0.59 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  96.63802454  DOF =  3447


[2023-07-27 20:08:54,539] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:08:54,540] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.54	  0.0359	    2.01	  0.1338	  0.3541
Line        489.15	  489.13	  489.21	    0.93	  0.0207	    4.24	  0.0943	  0.3037
Line        490.33	  490.28	  490.37	    1.50	  0.0300	    9.50	  0.1900	  0.4316
Line        492.39	  492.34	  492.42	    0.81	  0.0209	    2.99	  0.0767	  0.2735
Line        492.48	  492.45	  492.51	    0.84	  0.0323	    4.32	  0.1663	  0.4002
Line        493.88	  493.86	  493.91	    0.51	  0.0180	    2.28	  0.0813	  0.2802
Line        496.26	  496.24	  496.28	    0.84	  0.0367	    3.00	  0.1306	  0.3538
Line        496.61	  496.59	  496.65	    1.20	  0.0362	    7.62	  0.2308	  0.4733
Line        499.41	  499.39	  499.44	    0.81	  0.0281	    3.53	  0.1218	  0.3431
Line        500.19	  500.16	  500.22	    0.97	  0.0324	    5.46	  0.1820	  0.4197
Line        501.49	  501.45	  501.52	    1.27	  0.0326	    4.31	  0.1106	  0.3283
Line        

154

## Moog 2

In [11]:
line_list = "2"
code = "moog"
star_name = 'WASP-92'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
    

94


[2023-07-27 20:08:54,750] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:08:54,756] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:08:54,758] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:08:55,342] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:08:55,953] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:08:56,573] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:08:57,133] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:08:57,681] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:08:58,264] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:08:58,866] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:08:59,455] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:09:00,455] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:09:01,195] [INFO] [spectrum:estimate_snr:458

Generating: 6039 4.21 0.10 0.00 0.95 4.42 2.00 0.59 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  105.1282111  DOF =  3381
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 2  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 4.00 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 23.16 0.59 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  98.71907637  DOF =  3381
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 23.16262416  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 25.16 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 4.35 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 9.90 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 16.91 0.59 40000 l

[2023-07-27 20:13:22,890] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:13:22,891] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.52	  0.0403	    2.80	  0.2155	  0.4474
Line        489.15	  489.13	  489.21	    1.40	  0.0332	    7.87	  0.1874	  0.4278
Line        490.33	  490.28	  490.37	    1.32	  0.0313	    5.76	  0.1372	  0.3661
Line        492.39	  492.34	  492.42	    1.04	  0.0306	    6.42	  0.1888	  0.4282
Line        492.48	  492.45	  492.51	    0.93	  0.0266	    3.32	  0.0949	  0.3037
Line        493.88	  493.86	  493.91	    0.44	  0.0232	    1.59	  0.0835	  0.2817
Line        496.26	  496.24	  496.28	    0.75	  0.0328	    2.84	  0.1237	  0.3443
Line        496.61	  496.59	  496.65	    0.99	  0.0331	    5.08	  0.1694	  0.4049
Line        499.41	  499.39	  499.44	    0.91	  0.0303	    3.34	  0.1113	  0.3281
Line        500.19	  500.16	  500.22	    0.78	  0.0310	    3.40	  0.1361	  0.3618
Line        501.49	  501.45	  501.52	    0.88	  0.0244	    4.24	  0.1179	  0.3387
Line        

151

# Kepler-41

## Turbospectrum 2

In [12]:
line_list = "2"
code = "turbospectrum"
star_name = 'Kepler-41'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

78


[2023-07-27 20:13:23,254] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:13:23,262] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:13:23,268] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:13:23,965] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:13:24,801] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:13:25,450] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:13:26,115] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:13:26,791] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:13:27,460] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:13:28,121] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:13:28,813] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:13:29,452] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:13:30,039] [INFO] [spectrum:estimate_snr:458

Generating: 5766 4.32 0.21 0.00 0.47 3.42 2.00 0.62 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  1.817131502  DOF =  504
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 2  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 4.00 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 24.00 0.62 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  1.115103812  DOF =  504
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 23.99675738  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 26.00 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 30.79 0.62 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  1.087340197  DOF =  504
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vm

[2023-07-27 20:17:20,234] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:17:20,235] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	-9999.00	-9999.0000	-9999.00	-9999.0000	-9999.0000
Line        489.15	  489.13	  489.21	    0.04	  0.0033	    1.05	  0.0958	  0.2963
Line        490.33	  490.28	  490.37	    0.02	  0.0022	    0.62	  0.0776	  0.2626
Line        492.39	  492.34	  492.42	    0.00	  0.0010	    0.05	  0.0130	  0.1020
Line        492.48	  492.45	  492.51	    0.05	  0.0092	    1.67	  0.3332	  0.5269
Line        493.88	  493.86	  493.91	    0.00	  0.0005	    0.02	  0.0057	  0.0678
Line        496.26	  496.24	  496.28	    0.02	  0.0046	    0.20	  0.0512	  0.2024
Line        496.61	  496.59	  496.65	    0.00	  0.0012	    0.05	  0.0132	  0.1028
Line        499.41	  499.39	  499.44	    0.00	  0.0002	    0.02	  0.0031	  0.0506
Line        500.19	  500.16	  500.22	    0.03	  0.0056	    0.73	  0.1453	  0.3480
Line        501.49	  501.45	  501.52	    0.00	  0.0002	    0.01	  0.0031	  0.0496
Line  

162

## Synthe 2

In [13]:
line_list = "2"
code = "synthe"
star_name = 'Kepler-41'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

78


[2023-07-27 20:17:20,664] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:17:20,671] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:17:20,673] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:17:21,315] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:17:21,921] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:17:22,505] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:17:23,068] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:17:23,637] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:17:24,423] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:17:25,211] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:17:25,871] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:17:26,561] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:17:27,186] [INFO] [spectrum:estimate_snr:458

Generating: 5766 4.32 0.21 0.00 0.47 3.42 2.00 0.62 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  1.895581312  DOF =  491
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 2  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 4.00 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 23.81 0.62 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  1.336560314  DOF =  491
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 23.80843303  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 25.81 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 26.54 0.62 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  1.332292745  DOF =  491
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vm

[2023-07-27 20:21:16,567] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:21:16,568] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	-9999.00	-9999.0000	-9999.00	-9999.0000	-9999.0000
Line        489.15	  489.13	  489.21	    0.02	  0.0034	    0.52	  0.0870	  0.2732
Line        490.33	  490.28	  490.37	    0.02	  0.0029	    0.51	  0.0846	  0.2693
Line        492.39	  492.34	  492.42	    0.02	  0.0042	    0.59	  0.1467	  0.3426
Line        492.48	  492.45	  492.51	    0.00	  0.0000	    0.00	  0.0006	  0.0221
Line        493.88	  493.86	  493.91	-9999.00	-9999.0000	-9999.00	-9999.0000	-9999.0000
Line        496.26	  496.24	  496.28	    0.00	  0.0017	    0.09	  0.0442	  0.1717
Line        496.61	  496.59	  496.65	    0.01	  0.0057	    0.30	  0.1478	  0.3139
Line        499.41	  499.39	  499.44	    0.00	  0.0006	    0.03	  0.0098	  0.0856
Line        500.19	  500.16	  500.22	    0.00	  0.0005	    0.05	  0.0076	  0.0818
Line        501.49	  501.45	  501.52	    0.02	  0.0080	    0.58	  0.1940	  0.3815


155

## Moog 2

In [14]:
line_list = "2"
code = "moog"
star_name = 'Kepler-41'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
    

78


[2023-07-27 20:21:16,791] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:21:16,797] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:21:16,799] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:21:17,422] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:21:18,014] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:21:18,635] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:21:19,305] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:21:19,998] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:21:20,610] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:21:21,177] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:21:21,771] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:21:22,338] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:21:22,908] [INFO] [spectrum:estimate_snr:458

Generating: 5766 4.32 0.21 0.00 0.47 3.42 2.00 0.62 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  1.828443411  DOF =  547
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 2  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 4.00 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 25.32 0.62 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  1.145796319  DOF =  547
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 25.32078886  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 27.32 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 44.96 0.62 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  0.933010378  DOF =  547
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vm

[2023-07-27 20:22:57,424] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:22:57,425] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.00	  0.0000	    0.00	  0.0006	  0.0173
Line        489.15	  489.13	  489.21	    0.00	  0.0001	    0.01	  0.0019	  0.0401
Line        490.33	  490.28	  490.37	    0.00	  0.0002	    0.03	  0.0038	  0.0577
Line        492.39	  492.34	  492.42	    0.01	  0.0013	    0.23	  0.0329	  0.1696
Line        492.48	  492.45	  492.51	    0.03	  0.0038	    1.57	  0.1748	  0.3966
Line        493.88	  493.86	  493.91	    0.00	  0.0006	    0.03	  0.0140	  0.0965
Line        496.26	  496.24	  496.28	    0.00	  0.0003	    0.02	  0.0065	  0.0698
Line        496.61	  496.59	  496.65	    0.00	  0.0008	    0.06	  0.0158	  0.1123
Line        499.41	  499.39	  499.44	    0.00	  0.0001	    0.01	  0.0020	  0.0399
Line        500.19	  500.16	  500.22	    0.04	  0.0041	    1.14	  0.1266	  0.3376
Line        501.49	  501.45	  501.52	    0.01	  0.0020	    0.17	  0.0340	  0.1684
Line        

151

# CoRoT-4

## Turbospectrum 2

In [15]:
line_list = "2"
code = "turbospectrum"
star_name = 'CoRoT-4'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

[2023-07-27 20:22:57,567] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:22:57,573] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...


59


[2023-07-27 20:22:57,586] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:22:58,122] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:22:58,709] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:22:59,359] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:22:59,987] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:23:00,543] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:23:01,045] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:23:01,610] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:23:02,157] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:23:02,726] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:23:03,310] [INFO] [spectrum:estimate_snr:458]: SNR = 139.80
[2023-07-27 20:23:03,397] [INFO] [continuum:__fit_continuum:637]: Resampling spectrum to wave_step: 0.00100 nm (R ~ 472976)
[2023-07-27 20:23:05,995] [INFO] [continuum:__median_filter:307]: Median filter with step: 51
[2023-07-27 20:23:

Generating: 6238 4.55 0.18 0.00 1.23 4.49 2.00 0.58 56990 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  97743.21285  DOF =  6769
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 2  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 4.00 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 22.79 0.58 56990 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  41422.08787  DOF =  6769
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 22.78727533  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 24.79 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 42.89 0.58 56990 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  25737.97414  DOF =  6769
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vm

[2023-07-27 20:25:33,508] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:25:33,510] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    1.83	  0.0830	    0.19	  0.0087	  0.0913
Line        489.15	  489.13	  489.21	  114.64	  1.3812	    3.94	  0.0474	  0.2165
Line        490.33	  490.28	  490.37	   10.73	  0.1325	    0.28	  0.0034	  0.0583
Line        492.39	  492.34	  492.42	   53.44	  0.7031	    0.62	  0.0081	  0.0894
Line        492.48	  492.45	  492.51	   72.97	  1.1962	    0.82	  0.0135	  0.1153
Line        493.88	  493.86	  493.91	   84.54	  1.7613	    1.26	  0.0263	  0.1605
Line        496.26	  496.24	  496.28	    4.98	  0.1311	    0.05	  0.0014	  0.0371
Line        496.61	  496.59	  496.65	   47.42	  0.9298	    0.42	  0.0083	  0.0900
Line        499.41	  499.39	  499.44	   36.24	  0.7248	    0.26	  0.0052	  0.0713
Line        500.19	  500.16	  500.22	  106.59	  1.6919	    0.66	  0.0105	  0.1019
Line        501.49	  501.45	  501.52	  116.81	  1.7971	    0.65	  0.0101	  0.0995
Line        

163

## Synthe 2

In [16]:
line_list = "2"
code = "synthe"
star_name = 'CoRoT-4'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

59


[2023-07-27 20:25:33,656] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:25:33,666] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:25:33,671] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:25:34,226] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:25:34,815] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:25:35,396] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:25:35,959] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:25:36,508] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:25:37,034] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:25:37,598] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:25:38,147] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:25:38,715] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:25:39,299] [INFO] [spectrum:estimate_snr:458

Generating: 6238 4.55 0.18 0.00 1.23 4.49 2.00 0.58 56990 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  84435.27147  DOF =  6779
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 2  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 4.00 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 21.67 0.58 56990 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  34693.8544  DOF =  6779
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 21.67278186  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 23.67 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 38.43 0.58 56990 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  22117.05237  DOF =  6779
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmi

[2023-07-27 20:28:16,744] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:28:16,745] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	   78.97	  3.5896	    4.47	  0.2033	  0.4409
Line        489.15	  489.13	  489.21	   57.64	  0.7030	    1.71	  0.0209	  0.1436
Line        490.33	  490.28	  490.37	   14.94	  0.1845	    0.36	  0.0044	  0.0659
Line        492.39	  492.34	  492.42	   42.75	  0.5625	    0.69	  0.0090	  0.0945
Line        492.48	  492.45	  492.51	   24.89	  0.4081	    0.41	  0.0067	  0.0811
Line        493.88	  493.86	  493.91	   65.37	  1.3619	    0.91	  0.0189	  0.1360
Line        496.26	  496.24	  496.28	   11.15	  0.2934	    0.10	  0.0026	  0.0508
Line        496.61	  496.59	  496.65	   27.67	  0.5426	    0.23	  0.0044	  0.0659
Line        499.41	  499.39	  499.44	   10.22	  0.2043	    0.07	  0.0014	  0.0365
Line        500.19	  500.16	  500.22	   31.80	  0.5047	    0.17	  0.0026	  0.0510
Line        501.49	  501.45	  501.52	   76.45	  1.1945	    0.40	  0.0062	  0.0782
Line        

155

## Moog 2

In [17]:
line_list = "2"
code = "moog"
star_name = 'CoRoT-4'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

59


[2023-07-27 20:28:16,900] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:28:16,906] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:28:16,909] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:28:17,468] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:28:18,072] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:28:18,671] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:28:19,217] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:28:19,765] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:28:20,277] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:28:20,855] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:28:21,417] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:28:21,967] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:28:22,532] [INFO] [spectrum:estimate_snr:458

Generating: 6238 4.55 0.18 0.00 1.23 4.49 2.00 0.58 56990 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  100888.4584  DOF =  6783
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 2  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 4.00 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 23.08 0.58 56990 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  39652.67227  DOF =  6783
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 23.07743204  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 25.08 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 42.80 0.58 56990 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  26210.59627  DOF =  6783
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vm

[2023-07-27 20:29:10,672] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:29:10,674] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	   12.26	  0.5574	    0.96	  0.0434	  0.2038
Line        489.15	  489.13	  489.21	   40.96	  0.4995	    1.23	  0.0149	  0.1215
Line        490.33	  490.28	  490.37	    9.19	  0.1120	    0.22	  0.0026	  0.0510
Line        492.39	  492.34	  492.42	   40.39	  0.5314	    0.58	  0.0077	  0.0871
Line        492.48	  492.45	  492.51	   20.56	  0.3370	    0.34	  0.0056	  0.0741
Line        493.88	  493.86	  493.91	   45.36	  0.9450	    0.63	  0.0131	  0.1131
Line        496.26	  496.24	  496.28	    5.96	  0.1568	    0.06	  0.0016	  0.0393
Line        496.61	  496.59	  496.65	  100.60	  1.9726	    0.68	  0.0133	  0.1144
Line        499.41	  499.39	  499.44	   11.69	  0.2292	    0.08	  0.0015	  0.0384
Line        500.19	  500.16	  500.22	   33.64	  0.5340	    0.17	  0.0028	  0.0521
Line        501.49	  501.45	  501.52	   65.42	  1.0065	    0.32	  0.0050	  0.0701
Line        

153

# HD133131B

## Turbospectrum 2

In [18]:
line_list = "2"
code = "turbospectrum"
star_name = 'HD133131B'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

41


[2023-07-27 20:29:10,855] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:29:10,861] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:29:10,864] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:29:11,475] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:29:12,137] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:29:12,757] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:29:13,336] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:29:13,910] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:29:14,492] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:29:15,068] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:29:15,726] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:29:16,309] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:29:16,945] [INFO] [spectrum:estimate_snr:458

Generating: 5751 4.37 -0.28 0.11 0.90 3.29 2.00 0.59 74450 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  5355.392947  DOF =  6779
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 2  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 4.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.47 0.59 74450 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  4571.468453  DOF =  6779
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 0.4659884823  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 2.47 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.5

[2023-07-27 20:31:45,794] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:31:45,796] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    3.33	  0.1515	    0.33	  0.0151	  0.1202
Line        489.15	  489.13	  489.21	    3.64	  0.0444	    0.08	  0.0010	  0.0310
Line        490.33	  490.28	  490.37	    3.16	  0.0385	    0.07	  0.0009	  0.0292
Line        492.39	  492.34	  492.42	    7.09	  0.0932	    0.11	  0.0014	  0.0372
Line        492.48	  492.45	  492.51	    6.68	  0.1096	    0.11	  0.0018	  0.0416
Line        493.88	  493.86	  493.91	    3.19	  0.0665	    0.04	  0.0008	  0.0276
Line        496.26	  496.24	  496.28	    0.95	  0.0257	    0.01	  0.0003	  0.0172
Line        496.61	  496.59	  496.65	    5.82	  0.1141	    0.05	  0.0010	  0.0309
Line        499.41	  499.39	  499.44	    9.17	  0.1799	    0.04	  0.0008	  0.0272
Line        500.19	  500.16	  500.22	    8.55	  0.1379	    0.04	  0.0006	  0.0236
Line        501.49	  501.45	  501.52	   10.71	  0.1673	    0.04	  0.0007	  0.0253
Line        

/tmp/ipykernel_1748722/2750710842.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


142

## Synthe 2

In [19]:
line_list = "2"
code = "synthe"
star_name = 'HD133131B'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

41


[2023-07-27 20:31:45,958] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:31:45,963] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:31:45,965] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:31:46,527] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:31:47,126] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:31:47,716] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:31:48,283] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:31:48,828] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:31:49,389] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:31:49,948] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:31:50,519] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:31:51,082] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:31:51,654] [INFO] [spectrum:estimate_snr:458

Generating: 5751 4.37 -0.28 0.11 0.90 3.29 2.00 0.59 74450 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  5763.114304  DOF =  6779
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 2  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 4.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.39 0.59 74450 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  4930.596993  DOF =  6779
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 0.3924339332  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 2.39 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.5

[2023-07-27 20:34:38,206] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:34:38,208] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    3.28	  0.1492	    0.33	  0.0150	  0.1197
Line        489.15	  489.13	  489.21	    2.77	  0.0338	    0.06	  0.0008	  0.0274
Line        490.33	  490.28	  490.37	    2.45	  0.0299	    0.06	  0.0007	  0.0272
Line        492.39	  492.34	  492.42	    9.52	  0.1253	    0.14	  0.0018	  0.0423
Line        492.48	  492.45	  492.51	    5.55	  0.0910	    0.09	  0.0015	  0.0382
Line        493.88	  493.86	  493.91	    5.52	  0.1150	    0.06	  0.0013	  0.0355
Line        496.26	  496.24	  496.28	    0.65	  0.0175	    0.01	  0.0002	  0.0143
Line        496.61	  496.59	  496.65	    3.49	  0.0685	    0.03	  0.0006	  0.0244
Line        499.41	  499.39	  499.44	   10.98	  0.2152	    0.05	  0.0009	  0.0296
Line        500.19	  500.16	  500.22	   16.75	  0.2702	    0.07	  0.0011	  0.0325
Line        501.49	  501.45	  501.52	   17.34	  0.2709	    0.06	  0.0010	  0.0314
Line        

/tmp/ipykernel_1748722/3178073886.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


135

## Moog 2

In [20]:
line_list = "2"
code = "moog"
star_name = 'HD133131B'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

41


[2023-07-27 20:34:38,473] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:34:38,479] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:34:38,484] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:34:39,524] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:34:40,279] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:34:40,902] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:34:41,496] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:34:42,160] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:34:42,822] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:34:43,409] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:34:44,127] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:34:44,835] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:34:45,439] [INFO] [spectrum:estimate_snr:458

Generating: 5751 4.37 -0.28 0.11 0.90 3.29 2.00 0.59 74450 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  5363.185002  DOF =  6779
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 2  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 4.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.57 0.59 74450 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  4649.548059  DOF =  6779
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 0.5743499337  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 2.57 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.5

[2023-07-27 20:35:31,639] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:35:31,644] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    3.54	  0.1609	    0.35	  0.0161	  0.1241
Line        489.15	  489.13	  489.21	    3.62	  0.0442	    0.08	  0.0010	  0.0311
Line        490.33	  490.28	  490.37	    3.58	  0.0437	    0.08	  0.0010	  0.0312
Line        492.39	  492.34	  492.42	    8.24	  0.1084	    0.13	  0.0017	  0.0409
Line        492.48	  492.45	  492.51	    6.61	  0.1083	    0.11	  0.0018	  0.0415
Line        493.88	  493.86	  493.91	    3.51	  0.0731	    0.04	  0.0009	  0.0290
Line        496.26	  496.24	  496.28	    1.12	  0.0304	    0.01	  0.0004	  0.0186
Line        496.61	  496.59	  496.65	    6.11	  0.1197	    0.05	  0.0010	  0.0317
Line        499.41	  499.39	  499.44	    8.07	  0.1582	    0.03	  0.0007	  0.0258
Line        500.19	  500.16	  500.22	    8.40	  0.1355	    0.03	  0.0006	  0.0234
Line        501.49	  501.45	  501.52	   10.34	  0.1615	    0.04	  0.0006	  0.0248
Line        

/tmp/ipykernel_1748722/4166109291.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


133

# HATS-3

## Turbospectrum 2

In [21]:
line_list = "2"
code = "turbospectrum"
star_name = 'HATS-3'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

27


[2023-07-27 20:35:32,029] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:35:32,035] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:35:32,038] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:35:32,720] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:35:33,420] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:35:34,044] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:35:34,642] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:35:35,245] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:35:35,833] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:35:36,430] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:35:37,022] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:35:37,604] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:35:38,227] [INFO] [spectrum:estimate_snr:458

Generating: 6347 4.52 -0.02 0.01 1.41 5.03 2.00 0.56 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  26882.1565  DOF =  6773
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 2  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 4.00 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 25.59 0.56 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  10891.09269  DOF =  6773
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 25.5859739  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 27.59 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 42.44 0.56 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  8433.130512  DOF =  6773
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  

[2023-07-27 20:38:27,890] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:38:27,892] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.83	  0.0377	    0.01	  0.0004	  0.0204
Line        489.15	  489.13	  489.21	  178.35	  2.1750	    2.23	  0.0271	  0.1638
Line        490.33	  490.28	  490.37	   35.18	  0.4290	    0.42	  0.0051	  0.0712
Line        492.39	  492.34	  492.42	   54.44	  0.7163	    0.59	  0.0077	  0.0872
Line        492.48	  492.45	  492.51	   27.33	  0.4480	    0.27	  0.0044	  0.0655
Line        493.88	  493.86	  493.91	   51.64	  1.0758	    0.52	  0.0108	  0.1030
Line        496.26	  496.24	  496.28	   30.00	  0.8109	    0.24	  0.0064	  0.0791
Line        496.61	  496.59	  496.65	   11.69	  0.2292	    0.11	  0.0021	  0.0458
Line        499.41	  499.39	  499.44	   16.40	  0.3216	    0.14	  0.0028	  0.0526
Line        500.19	  500.16	  500.22	   18.99	  0.3014	    0.16	  0.0025	  0.0499
Line        501.49	  501.45	  501.52	  531.19	  8.2998	    2.58	  0.0403	  0.1993
Line        

176

## Synthe 2

In [22]:
line_list = "2"
code = "synthe"
star_name = 'HATS-3'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

27


[2023-07-27 20:38:28,188] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:38:28,194] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:38:28,196] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:38:28,775] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:38:29,361] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:38:29,942] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:38:30,498] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:38:31,058] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:38:31,621] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:38:32,173] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:38:32,722] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:38:33,281] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:38:33,852] [INFO] [spectrum:estimate_snr:458

Generating: 6347 4.52 -0.02 0.01 1.41 5.03 2.00 0.56 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  26696.95898  DOF =  6792
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 2  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 4.00 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 25.46 0.56 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  10919.39436  DOF =  6792
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 25.45909857  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 27.46 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 42.11 0.56 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  8492.336331  DOF =  6792
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554

[2023-07-27 20:41:18,616] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:41:18,619] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.55	  0.0248	    0.01	  0.0003	  0.0163
Line        489.15	  489.13	  489.21	  180.00	  2.1687	    2.14	  0.0258	  0.1596
Line        490.33	  490.28	  490.37	   36.17	  0.4411	    0.41	  0.0051	  0.0707
Line        492.39	  492.34	  492.42	   54.63	  0.7095	    0.57	  0.0075	  0.0858
Line        492.48	  492.45	  492.51	   28.44	  0.4662	    0.27	  0.0044	  0.0658
Line        493.88	  493.86	  493.91	   52.48	  1.0933	    0.52	  0.0109	  0.1032
Line        496.26	  496.24	  496.28	   29.93	  0.7875	    0.24	  0.0062	  0.0779
Line        496.61	  496.59	  496.65	   11.51	  0.2214	    0.11	  0.0021	  0.0450
Line        499.41	  499.39	  499.44	   15.19	  0.3038	    0.13	  0.0027	  0.0513
Line        500.19	  500.16	  500.22	   17.54	  0.2830	    0.15	  0.0024	  0.0485
Line        501.49	  501.45	  501.52	  530.36	  8.1594	    2.60	  0.0400	  0.1986
Line        

171

## Moog 2

In [23]:
line_list = "2"
code = "moog"
star_name = 'HATS-3'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
  

27


[2023-07-27 20:41:19,034] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:41:19,042] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:41:19,046] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:41:20,216] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:41:21,071] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:41:21,748] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:41:22,615] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:41:23,387] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:41:24,076] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:41:24,710] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:41:25,349] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:41:25,938] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:41:26,555] [INFO] [spectrum:estimate_snr:458

Generating: 6347 4.52 -0.02 0.01 1.41 5.03 2.00 0.56 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  26470.77525  DOF =  6779
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 2  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 4.00 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 25.16 0.56 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  10737.34021  DOF =  6779
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 25.16233583  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 27.16 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 41.34 0.56 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  8315.987647  DOF =  6779
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554

[2023-07-27 20:42:15,691] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:42:15,693] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.64	  0.0290	    0.01	  0.0003	  0.0179
Line        489.15	  489.13	  489.21	  161.72	  1.9484	    1.98	  0.0239	  0.1535
Line        490.33	  490.28	  490.37	   31.04	  0.3786	    0.37	  0.0045	  0.0663
Line        492.39	  492.34	  492.42	   46.48	  0.6116	    0.50	  0.0065	  0.0802
Line        492.48	  492.45	  492.51	   28.48	  0.4669	    0.27	  0.0044	  0.0660
Line        493.88	  493.86	  493.91	   49.57	  1.0327	    0.50	  0.0104	  0.1007
Line        496.26	  496.24	  496.28	   22.76	  0.5988	    0.18	  0.0049	  0.0688
Line        496.61	  496.59	  496.65	   11.88	  0.2329	    0.11	  0.0022	  0.0464
Line        499.41	  499.39	  499.44	   15.33	  0.3005	    0.14	  0.0027	  0.0512
Line        500.19	  500.16	  500.22	   19.96	  0.3169	    0.17	  0.0027	  0.0515
Line        501.49	  501.45	  501.52	  464.36	  7.1440	    2.34	  0.0361	  0.1885
Line        

170

# WASP-87A


## Turbospectrum 2

In [24]:
line_list = "2"
code = "turbospectrum"
star_name = 'WASP-87A'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

58


[2023-07-27 20:42:15,895] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:42:15,902] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:42:15,907] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:42:16,551] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:42:17,198] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:42:17,812] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:42:18,402] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:42:18,964] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:42:19,534] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:42:20,113] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:42:20,681] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:42:21,278] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:42:21,875] [INFO] [spectrum:estimate_snr:458

Generating: 6529 4.51 -0.28 0.11 2.10 5.95 2.00 0.54 107200 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  74997.34044  DOF =  6783
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 2  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 4.00 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 22.62 0.54 107200 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  27090.1341  DOF =  6783
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 22.61625984  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 24.62 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 40.21 0.54 107200 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  17204.56515  DOF =  6783
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.

[2023-07-27 20:45:05,591] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:45:05,593] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    5.43	  0.2467	    0.56	  0.0252	  0.1554
Line        489.15	  489.13	  489.21	   35.29	  0.4252	    1.33	  0.0161	  0.1261
Line        490.33	  490.28	  490.37	    3.67	  0.0447	    0.11	  0.0013	  0.0364
Line        492.39	  492.34	  492.42	   11.55	  0.1520	    0.25	  0.0032	  0.0564
Line        492.48	  492.45	  492.51	    5.90	  0.0968	    0.12	  0.0020	  0.0441
Line        493.88	  493.86	  493.91	   27.11	  0.5649	    0.47	  0.0098	  0.0982
Line        496.26	  496.24	  496.28	  148.17	  3.8993	    1.37	  0.0360	  0.1873
Line        496.61	  496.59	  496.65	   11.48	  0.2250	    0.13	  0.0025	  0.0497
Line        499.41	  499.39	  499.44	   13.07	  0.2562	    0.11	  0.0021	  0.0454
Line        500.19	  500.16	  500.22	   69.27	  1.0995	    0.40	  0.0063	  0.0787
Line        501.49	  501.45	  501.52	 1399.40	 21.5293	    4.78	  0.0735	  0.2691
Line        

178

## Synthe 2

In [25]:
line_list = "2"
code = "synthe"
star_name = 'WASP-87A'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

58


[2023-07-27 20:45:05,757] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:45:05,768] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:45:05,771] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:45:06,407] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:45:07,096] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:45:07,796] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:45:08,438] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:45:09,026] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:45:09,617] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:45:10,255] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:45:10,951] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:45:11,744] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:45:12,448] [INFO] [spectrum:estimate_snr:458

Generating: 6529 4.51 -0.28 0.11 2.10 5.95 2.00 0.54 107200 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  73387.90538  DOF =  6781
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 2  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 4.00 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 22.60 0.54 107200 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  26552.47356  DOF =  6781
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 22.60348624  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 24.60 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 40.13 0.54 107200 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  16956.34181  DOF =  6781
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0

[2023-07-27 20:48:07,819] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:48:07,823] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    5.76	  0.2617	    0.59	  0.0268	  0.1600
Line        489.15	  489.13	  489.21	   34.44	  0.4150	    1.30	  0.0157	  0.1246
Line        490.33	  490.28	  490.37	    3.67	  0.0447	    0.11	  0.0013	  0.0365
Line        492.39	  492.34	  492.42	   11.38	  0.1478	    0.24	  0.0031	  0.0557
Line        492.48	  492.45	  492.51	    5.71	  0.0936	    0.12	  0.0019	  0.0434
Line        493.88	  493.86	  493.91	   27.19	  0.5665	    0.47	  0.0099	  0.0984
Line        496.26	  496.24	  496.28	  147.77	  3.8886	    1.37	  0.0360	  0.1872
Line        496.61	  496.59	  496.65	   11.49	  0.2253	    0.13	  0.0025	  0.0497
Line        499.41	  499.39	  499.44	   12.78	  0.2506	    0.10	  0.0021	  0.0449
Line        500.19	  500.16	  500.22	   69.04	  1.0959	    0.40	  0.0063	  0.0786
Line        501.49	  501.45	  501.52	 1405.78	 21.6274	    4.81	  0.0740	  0.2700
Line        

172

## Moog 2

In [26]:
line_list = "2"
code = "moog"
star_name = 'WASP-87A'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
  

58


[2023-07-27 20:48:07,971] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:48:07,980] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:48:07,984] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:48:08,553] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:48:09,167] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:48:09,785] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:48:10,402] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:48:11,056] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:48:11,693] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:48:12,301] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:48:12,931] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:48:13,886] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:48:15,254] [INFO] [spectrum:estimate_snr:458

Generating: 6529 4.51 -0.28 0.11 2.10 5.95 2.00 0.54 107200 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  76400.00305  DOF =  6788
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 2  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 4.00 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 22.58 0.54 107200 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  27467.14199  DOF =  6788
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 22.57881695  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 24.58 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 40.12 0.54 107200 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  17444.98695  DOF =  6788
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0

[2023-07-27 20:49:06,987] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:49:06,989] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    5.70	  0.2591	    0.58	  0.0265	  0.1591
Line        489.15	  489.13	  489.21	   29.64	  0.3571	    1.12	  0.0135	  0.1154
Line        490.33	  490.28	  490.37	    2.72	  0.0331	    0.08	  0.0010	  0.0313
Line        492.39	  492.34	  492.42	    9.60	  0.1263	    0.20	  0.0027	  0.0514
Line        492.48	  492.45	  492.51	    5.37	  0.0881	    0.11	  0.0018	  0.0419
Line        493.88	  493.86	  493.91	   27.06	  0.5637	    0.47	  0.0098	  0.0980
Line        496.26	  496.24	  496.28	  148.24	  3.9010	    1.37	  0.0360	  0.1874
Line        496.61	  496.59	  496.65	   11.37	  0.2230	    0.13	  0.0025	  0.0494
Line        499.41	  499.39	  499.44	   12.78	  0.2557	    0.11	  0.0021	  0.0454
Line        500.19	  500.16	  500.22	   68.53	  1.0878	    0.39	  0.0063	  0.0786
Line        501.49	  501.45	  501.52	 1374.10	 21.4703	    4.72	  0.0738	  0.2695
Line        

171

# HATS-10

## Turbospectrum 2

In [27]:
line_list = "2"
code = "turbospectrum"
star_name = 'HATS-10'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

48


[2023-07-27 20:49:07,389] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:49:07,401] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:49:07,406] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:49:08,088] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:49:08,805] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:49:09,472] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:49:10,120] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:49:10,920] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:49:11,660] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:49:12,353] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:49:13,021] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:49:13,746] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:49:14,452] [INFO] [spectrum:estimate_snr:458

Generating: 6167 4.51 0.40 0.00 1.45 4.28 2.00 0.59 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  10768.76407  DOF =  6773
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 2  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.00 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 24.52 0.59 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  8238.15728  DOF =  6773
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 24.52096094  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 26.52 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 29.31 0.59 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  8187.387871  DOF =  6773
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic =

[2023-07-27 20:52:13,264] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:52:13,265] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    1.59	  0.0722	    0.16	  0.0071	  0.0822
Line        489.15	  489.13	  489.21	   58.83	  0.7088	    2.92	  0.0352	  0.1864
Line        490.33	  490.28	  490.37	   15.52	  0.1893	    1.43	  0.0174	  0.1312
Line        492.39	  492.34	  492.42	   16.13	  0.2122	    1.38	  0.0181	  0.1336
Line        492.48	  492.45	  492.51	  116.08	  1.9029	    4.72	  0.0774	  0.2759
Line        493.88	  493.86	  493.91	   14.34	  0.2927	    0.90	  0.0183	  0.1340
Line        496.26	  496.24	  496.28	    2.98	  0.0805	    0.24	  0.0065	  0.0794
Line        496.61	  496.59	  496.65	    7.57	  0.1484	    0.64	  0.0126	  0.1110
Line        499.41	  499.39	  499.44	    8.46	  0.1692	    0.67	  0.0135	  0.1149
Line        500.19	  500.16	  500.22	    9.57	  0.1518	    0.75	  0.0119	  0.1081
Line        501.49	  501.45	  501.52	   65.00	  1.0156	    2.97	  0.0465	  0.2139
Line        

161

## Synthe 2

In [28]:
line_list = "2"
code = "synthe"
star_name = 'HATS-10'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

48


[2023-07-27 20:52:13,679] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:52:13,688] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:52:13,693] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:52:14,380] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:52:15,074] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:52:15,754] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:52:16,407] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:52:17,051] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:52:17,817] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:52:18,494] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:52:19,121] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:52:19,794] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:52:20,477] [INFO] [spectrum:estimate_snr:458

Generating: 6167 4.51 0.40 0.00 1.45 4.28 2.00 0.59 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  6918.127112  DOF =  6777
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 2  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.00 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.94 0.59 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  6879.263828  DOF =  6777
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 4.939809104  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 6.94 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.73 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.73 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.73 0.59 48000 lo

[2023-07-27 20:55:14,661] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:55:14,663] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.59	  0.0257	    0.05	  0.0023	  0.0470
Line        489.15	  489.13	  489.21	    2.40	  0.0289	    0.16	  0.0019	  0.0436
Line        490.33	  490.28	  490.37	    4.34	  0.0529	    0.32	  0.0039	  0.0617
Line        492.39	  492.34	  492.42	    2.75	  0.0361	    0.18	  0.0023	  0.0478
Line        492.48	  492.45	  492.51	    3.78	  0.0620	    0.29	  0.0048	  0.0688
Line        493.88	  493.86	  493.91	    1.38	  0.0288	    0.09	  0.0019	  0.0433
Line        496.26	  496.24	  496.28	    1.64	  0.0432	    0.12	  0.0031	  0.0551
Line        496.61	  496.59	  496.65	    1.84	  0.0361	    0.14	  0.0027	  0.0510
Line        499.41	  499.39	  499.44	    1.62	  0.0318	    0.11	  0.0021	  0.0457
Line        500.19	  500.16	  500.22	    2.53	  0.0402	    0.17	  0.0027	  0.0520
Line        501.49	  501.45	  501.52	    1.92	  0.0300	    0.13	  0.0021	  0.0453
Line        

154

## Moog 2

In [29]:
line_list = "2"
code = "moog"
star_name = 'HATS-10'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

48


[2023-07-27 20:55:15,023] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:55:15,030] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:55:15,034] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:55:15,735] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:55:16,464] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:55:17,088] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:55:17,727] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:55:18,335] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:55:18,981] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:55:19,715] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:55:20,422] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:55:21,047] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:55:21,697] [INFO] [spectrum:estimate_snr:458

Generating: 6167 4.51 0.40 0.00 1.45 4.28 2.00 0.59 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  6950.421477  DOF =  6777
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 2  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.00 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.98 0.59 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  6909.611938  DOF =  6777
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 4.979232694  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 6.98 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.74 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.74 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.74 0.59 48000 lo

[2023-07-27 20:56:08,356] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:56:08,358] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.76	  0.0345	    0.06	  0.0029	  0.0528
Line        489.15	  489.13	  489.21	    2.44	  0.0294	    0.16	  0.0020	  0.0439
Line        490.33	  490.28	  490.37	    3.59	  0.0438	    0.26	  0.0031	  0.0555
Line        492.39	  492.34	  492.42	    2.12	  0.0279	    0.14	  0.0019	  0.0428
Line        492.48	  492.45	  492.51	    4.49	  0.0737	    0.35	  0.0058	  0.0755
Line        493.88	  493.86	  493.91	    1.73	  0.0361	    0.12	  0.0025	  0.0493
Line        496.26	  496.24	  496.28	    1.17	  0.0308	    0.09	  0.0023	  0.0471
Line        496.61	  496.59	  496.65	    2.34	  0.0458	    0.17	  0.0033	  0.0571
Line        499.41	  499.39	  499.44	    1.68	  0.0329	    0.11	  0.0022	  0.0468
Line        500.19	  500.16	  500.22	    1.61	  0.0260	    0.11	  0.0018	  0.0424
Line        501.49	  501.45	  501.52	    1.76	  0.0275	    0.12	  0.0019	  0.0434
Line        

152

# HATS-56

## Turbospectrum 2

In [30]:
line_list = "2"
code = "turbospectrum"
star_name = 'HATS-56'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

71


[2023-07-27 20:56:08,630] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:56:08,635] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:56:08,638] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:56:09,425] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:56:10,192] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:56:10,916] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:56:11,645] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:56:12,351] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:56:13,347] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:56:14,252] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:56:14,954] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:56:15,601] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:56:16,325] [INFO] [spectrum:estimate_snr:458

Generating: 6594 3.98 0.25 0.00 1.75 7.37 2.00 0.55 115000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  2882.873582  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 2  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 4.00 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.37 0.55 115000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  2859.82941  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 1.370381712  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 3.37 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.03 0.55 115000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  2855.587686  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  

[2023-07-27 20:59:38,046] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 20:59:38,047] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    4.68	  0.2128	    0.02	  0.0010	  0.0304
Line        489.15	  489.13	  489.21	   24.27	  0.2959	    0.15	  0.0019	  0.0430
Line        490.33	  490.28	  490.37	   21.28	  0.2595	    0.14	  0.0017	  0.0408
Line        492.39	  492.34	  492.42	   53.75	  0.7073	    0.22	  0.0029	  0.0531
Line        492.48	  492.45	  492.51	   11.71	  0.1920	    0.08	  0.0012	  0.0348
Line        493.88	  493.86	  493.91	   10.22	  0.2129	    0.06	  0.0012	  0.0341
Line        496.26	  496.24	  496.28	    7.33	  0.1929	    0.04	  0.0012	  0.0335
Line        496.61	  496.59	  496.65	   13.59	  0.2666	    0.07	  0.0013	  0.0360
Line        499.41	  499.39	  499.44	   11.38	  0.2276	    0.06	  0.0011	  0.0333
Line        500.19	  500.16	  500.22	   19.73	  0.3132	    0.11	  0.0017	  0.0406
Line        501.49	  501.45	  501.52	   21.65	  0.3331	    0.10	  0.0016	  0.0398
Line        

161

## Synthe 2

In [31]:
line_list = "2"
code = "synthe"
star_name = 'HATS-56'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
   

71


[2023-07-27 20:59:38,270] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 20:59:38,278] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 20:59:38,281] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 20:59:38,982] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 20:59:39,956] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 20:59:40,762] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 20:59:41,530] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 20:59:42,202] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 20:59:42,874] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 20:59:43,563] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 20:59:44,262] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 20:59:45,020] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 20:59:45,715] [INFO] [spectrum:estimate_snr:458

Generating: 6594 3.98 0.25 0.00 1.75 7.37 2.00 0.55 115000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  2999.714646  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 2  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 4.00 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.62 0.55 115000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  2990.246447  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 1.618459695  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 3.62 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.45 0.55 115000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  2988.497829  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175 

[2023-07-27 21:02:41,819] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 21:02:41,823] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    4.55	  0.2070	    0.02	  0.0010	  0.0303
Line        489.15	  489.13	  489.21	   22.74	  0.2774	    0.15	  0.0018	  0.0422
Line        490.33	  490.28	  490.37	   20.02	  0.2442	    0.13	  0.0016	  0.0401
Line        492.39	  492.34	  492.42	   64.59	  0.8499	    0.26	  0.0034	  0.0582
Line        492.48	  492.45	  492.51	   10.52	  0.1724	    0.07	  0.0011	  0.0331
Line        493.88	  493.86	  493.91	   13.47	  0.2806	    0.07	  0.0015	  0.0389
Line        496.26	  496.24	  496.28	    7.17	  0.1886	    0.04	  0.0011	  0.0335
Line        496.61	  496.59	  496.65	   12.37	  0.2425	    0.06	  0.0012	  0.0348
Line        499.41	  499.39	  499.44	   12.63	  0.2527	    0.06	  0.0013	  0.0356
Line        500.19	  500.16	  500.22	   23.31	  0.3701	    0.12	  0.0020	  0.0441
Line        501.49	  501.45	  501.52	   25.28	  0.3889	    0.12	  0.0019	  0.0431
Line        

154

## Moog 2

In [32]:
line_list = "2"
code = "moog"
star_name = 'HATS-56'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
   

71


[2023-07-27 21:02:42,046] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 21:02:42,052] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 21:02:42,055] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 21:02:42,722] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 21:02:43,476] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 21:02:44,191] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 21:02:44,890] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 21:02:45,560] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 21:02:46,209] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 21:02:46,894] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 21:02:47,759] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 21:02:48,478] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 21:02:49,162] [INFO] [spectrum:estimate_snr:458

Generating: 6594 3.98 0.25 0.00 1.75 7.37 2.00 0.55 115000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  3033.040364  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 2  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 4.00 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.91 0.55 115000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  3032.25596  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 1.91268989  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 3.91 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.88 0.55 115000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  3032.10264  DOF =  6785
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
 

[2023-07-27 21:03:36,143] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 21:03:36,145] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    4.64	  0.2108	    0.02	  0.0009	  0.0300
Line        489.15	  489.13	  489.21	   25.65	  0.3128	    0.17	  0.0020	  0.0449
Line        490.33	  490.28	  490.37	   22.01	  0.2684	    0.14	  0.0017	  0.0413
Line        492.39	  492.34	  492.42	  108.99	  1.4340	    0.47	  0.0061	  0.0779
Line        492.48	  492.45	  492.51	   10.87	  0.1782	    0.07	  0.0011	  0.0332
Line        493.88	  493.86	  493.91	   10.80	  0.2250	    0.06	  0.0013	  0.0350
Line        496.26	  496.24	  496.28	    7.58	  0.1996	    0.04	  0.0012	  0.0340
Line        496.61	  496.59	  496.65	   14.42	  0.2828	    0.07	  0.0014	  0.0370
Line        499.41	  499.39	  499.44	   12.54	  0.2507	    0.06	  0.0012	  0.0350
Line        500.19	  500.16	  500.22	   19.83	  0.3147	    0.11	  0.0017	  0.0406
Line        501.49	  501.45	  501.52	   22.04	  0.3391	    0.11	  0.0016	  0.0401
Line        

152

# KPS-1

## Turbospectrum 2

## Synthe 2

## Moog 2

# Kepler-40

## Turbospectrum 2

## Synthe 2

## Moog 2

# Kepler-435

## Turbospectrum 2

## Synthe 2

## Moog 2